In [1]:
import tensorflow as tf

Creating 2 floating point tensors (constant nodes)

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # it is tf.float32 implicitly
print(node1, node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


Para evaluar los nodos, hay que lanzar el grafo computacional en una session:

In [4]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


Las operaciones entre tensores (como por ejemplo los tensores constantes ya definidos) también son nodos del grafo. Definimos un nodo "suma":

In [5]:
node3 = tf.add(node1, node2)
print("node3:",node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


En lugar de constantes, podemos definir variables, como placeholders:

In [6]:
a=tf.placeholder(tf.float32)
b=tf.placeholder(tf.float32)
adder_node = a + b  # + es un shortcut de tf.add(a,b)

Los argumentos, o valores de las variables, los pasamos con un diccionario:

In [7]:
print(sess.run(adder_node, {a:3, b:4.5}))
print(sess.run(adder_node, {a:[1,3], b:[2,4]}))

7.5
[ 3.  7.]


Agregamos otra operación al grafo:

In [8]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a:3, b:4.5}))

22.5


Los parámetro de un modelo de "machine learning", como son los pesos y bias de una red neuronal (W,b), se llaman parámetros entrenables, y se definen en tensor flow como Variable. Con Variables y Placeholders definimos el modelo:

In [10]:
W = tf.Variable([.3],dtype=tf.float32)
b = tf.Variable([-.3],dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

Las constantes se inicializan al llamar tf.constant, pero no así las variables al llamar tf.Variable. Para inicializar las variables de un programa de TensorFlow, hay que utilizar una función especial:

In [11]:
init = tf.global_variables_initializer()
sess.run(init)

Las variables se inicializan al llamar la función sess.run.
Como x es un placeholder, podemos evaluar el modelo para varios valores de x al mismo tiempo:

In [12]:
print(sess.run(linear_model,{x:[1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


Definimos la variable que contiene los target para entrenar el modelo, y, y la función de coste, como suma cuadrática de residuos entre el resultado del modelo y el target (y):

In [13]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)  #reduce_sum: suma a lo largo de una dimensión del tensor
print(sess.run(loss,{x:[1, 2, 3, 4], y:[0, -1, -2, -3]}))

23.66


Podemos calcular el resultado para otros valores de los parámetros W y b. Para cambiar estos valores se utiliza la función tf.assign:

In [14]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


Ahora aplicaremos gradient descent para obtener estos parámetros que optimizan el modelo. Vamos a hacer un entrenamiento con la API tf.train

In [18]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init) # reseteamos los parámetros a sus valores por defecto
for i in range(1000):
    sess.run(train, {x:[1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


Ponemos en un bloque todo el proceso anterior:

In [19]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## Libreria tf.estimator

Esta libreria contiene el ecosistema de machine learning: entrenamiento, evaluación, manejo de datos. Contiene varios modelos definidos. Veamos ejemplo:

In [21]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir='./output')

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_session_config': None, '_model_dir': './output', '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./output\model.ckpt.
INFO:tensorflow:loss = 9.0, step = 1
INFO:tensorflow:global_step/sec: 1492.54
INFO:tensorflow:loss = 0.141233, step = 101 (0.069 sec)
INFO:tensorflow:global_step/sec: 1785.73
INFO:tensorflow:loss = 0.0212379, step = 201 (0.055 sec)
INFO:tensorflow:global_step/sec: 1923.07
INFO:tensorflow:loss = 0.0020579, step = 301 (0.052 sec)
INFO:tensorflow:global_step/sec: 1886.78
INFO:tensorflow:loss = 0.000126658, step = 401 (0.054 sec)
INFO:tensorflow:global_step/sec: 1818.17
INFO:tensorflow:loss = 9.74265e-05, step = 501 (0.054 sec)
INFO:tensorflow:global_step

### Custom model

Finalmente utilizaremos tf.estimator con un modelo definido por el usuario, que puede hacerse como objeto de la clase tf.estimator.Estimator. En este caso definiremos un modelo de regresión lineal, que es el mismo que hemos utilizado de la propia librería, pero podríamos definir cualquier modelo:

In [28]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W * features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_session_config': None, '_model_dir': 'C:\\Users\\C2416~1.GIN\\AppData\\Local\\Temp\\tmpts62cj4l', '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\C2416~1.GIN\AppData\Local\Temp\tmpts62cj4l\model.ckpt.
INFO:tensorflow:loss = 2.18021338667, step = 1
INFO:tensorflow:global_step/sec: 1666.89
INFO:tensorflow:loss = 0.011779787422, step = 101 (0.062 sec)
INFO:tensorflow:global_step/sec: 1639.32
INFO:tensorflow:loss = 0.00198463306561, step = 201 (0.060 sec)
INFO:tensorflow:global_step/sec: 1694.94
INFO:tensorflow:loss = 0.000475324594241, step = 301 (0.060 sec)
INFO:tensorflow:global_step/sec: 1562.43
INFO:tensorflow:loss = 2.41956027215e-05, step = 401 (0.064 sec